In [1]:
import numpy as np
import scipy
from astropy.io import fits
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.cosmology import Planck18 as cosmo
from functools import partial
import time

In [2]:
path_halo = '/sps/lsst/groups/clusters/cluster_comparison_project/after_matching/cosmoDC2_pywazp.DC2.tpz.T500k.pointEstimateMags/zband/member_matching/fshare_0.1_pref_more_massive/cat1.fits'

path_amas = '/sps/lsst/groups/clusters/cluster_comparison_project/after_matching/cosmoDC2_pywazp.DC2.tpz.T500k.pointEstimateMags/zband/member_matching/fshare_0.1_pref_more_massive/cat2.fits'

t = Table.read(path_amas)

def table(path):
    with fits.open(path) as hdul:
        table = Table(hdul[1].data)
    return table

t_halo = table(path_halo)
t_amas = table(path_amas)
t_amas_match = t_amas[t_amas["mt_cross"] !='']

M = []
for id in t_amas_match["mt_cross"]:
    ligne = t_halo[t_halo["halo_id"] == id]
    M.append(ligne["m200c"][0])
t_amas_match["M"] = M
t_amas_match = t_amas_match[t_amas_match["snr"]>6]

def tableau_coupure_n200(coupure):
    return t_amas_match[t_amas_match["n200"]>coupure]

Mp, zp = 10**14.3, 0.5
list_coupures = np.linspace(20, 80, 13)
print("liste des coupures :", list_coupures)

liste des coupures : [20. 25. 30. 35. 40. 45. 50. 55. 60. 65. 70. 75. 80.]


In [3]:
#X est le catalogue
def _ln_L(params, X):
    A, B, C, D, E, F = params
    #A, B, D = params
    C, E, F = 0, 0, 0  # on fixe 3 paramètr
    pop, M, z = np.log(X["n200"]), X["M"], X["zp"]
    mean_alpha = A + B*np.log(M/Mp) + C*np.log((1+z)/(1+zp))
    sigma = D + E*np.log(M/Mp) + F*np.log((1+z)/(1+zp))
    sigma = np.clip(sigma, 1e-5, None)
    #print("sigma max/min :", max(sigma), min(sigma))
    #log-Vraissemblance
    ln_L = - sum(np.log(sigma*(2*np.pi)**0.5)) - sum(0.5*((pop-mean_alpha)/sigma)**2)
    #print(-ln_L)
    return -ln_L

In [16]:
# Domaine de E exploré :
A = 4.264
B = 0.268
C = np.arange(-4, 4, 0.1)
D = 0.259
E = np.arange(-4, 4, 0.1)
F = np.arange(-4, 4, 0.1)

X = t_amas_match[t_amas_match["n200"]>50]


In [20]:
N = len(C)
L = np.zeros((N, N, N), dtype = float)
T = time.time()
for i in range(N):
    if (100*i/N) % 5 == 0 and i != 0:
        dt = time.time() - T
        t_restant = (1-i/N)*20*dt
        print("progression : "+str(round(100*i/N, 0))+" %")
        print("Temps restant estimé : "+str(int(t_restant//60))+"m "+str(int(t_restant%60))+"s")
        T = time.time()
    for j in range(N):
        for k in range(N):
            L[i][j][k] = _ln_L(params = (A, B, C[i], D, E[j], F[k]), X = X)

progression : 5.0 %
Temps restant estimé : 10m 45s
progression : 10.0 %
Temps restant estimé : 10m 8s
progression : 15.0 %
Temps restant estimé : 9m 35s
progression : 20.0 %
Temps restant estimé : 9m 8s
progression : 25.0 %
Temps restant estimé : 8m 38s
progression : 30.0 %
Temps restant estimé : 7m 55s
progression : 35.0 %
Temps restant estimé : 7m 24s
progression : 40.0 %
Temps restant estimé : 6m 54s
progression : 45.0 %
Temps restant estimé : 6m 12s
progression : 50.0 %
Temps restant estimé : 5m 40s
progression : 55.0 %
Temps restant estimé : 5m 8s
progression : 60.0 %
Temps restant estimé : 4m 30s
progression : 65.0 %
Temps restant estimé : 3m 57s
progression : 70.0 %
Temps restant estimé : 3m 26s
progression : 75.0 %
Temps restant estimé : 2m 48s
progression : 80.0 %
Temps restant estimé : 2m 15s
progression : 85.0 %
Temps restant estimé : 1m 43s
progression : 90.0 %
Temps restant estimé : 1m 7s
progression : 95.0 %
Temps restant estimé : 0m 34s


In [23]:
mini = (L[0][0][0], (0, 0, 0))
for i in range(N):
    for j in range(N):
        for k in range(N):
            l = L[i][j][k]
            if l < mini[0]:
                mini = (l, (i, j, k))
i, j, k = mini[1]                
print("A =", A)
print("B =", B)
print("C =", C[i])
print("D =", D)
print("E =", E[j])
print("F =", F[k])

A = 4.264
B = 0.268
C = -4.0
D = 0.259
E = -4.0
F = -4.0


In [24]:
print(L[i][j][k])
print(L.min())

59.52801419215638
59.52801419215638
